Reference:  
https://huggingface.co/docs/transformers/v4.17.0/en/tasks/sequence_classification  
https://huggingface.co/course/chapter3/3?fw=pt

## Preparing Variables

In [1]:
# Optional
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
seq_length = 256
pad_approach = "max_length"
bert_huggingface = "indolem/indobertweet-base-uncased" #uncased because we're using lower casing for all text data
num_epoch = 3

files = {
    "train" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/train.csv",
    "test" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/test.csv"
}

save_model_path = "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet"

load_model = True
load_path = "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet"

## Install Prerequisite Library

In [3]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 37.8 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.0 MB/s 
     |████████████████████████████████| 140 kB 51.3 MB/s 
     |████████████████████████████████| 212 kB 44.3 MB/s 
     |████████████████████████████████| 1.1 MB 46.9 MB/s 
     |████████████████████████████████| 127 kB 59.0 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 60.0 MB/s 
     |█████████████

## Preparing Datasets for Training
The Datasets must consist of "text" column name and "labels" column name
  
And for the "label" we need to process 0 as negative sentiment and 1 as positive sentiment

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files=files)

Using custom data configuration default-4d4b438ee4bf66a5


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4d4b438ee4bf66a5/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## Tokenizing Datasets for Training and Evaluation

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(load_path if load_model else bert_huggingface, use_fast=True)

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def tokenize_text(dataset_row):
    return tokenizer(str(dataset_row["text"]), max_length=seq_length,
                     truncation=True, padding="max_length")
    
    # # Extract mapping between new and old indices
    # sample_map = tokenize.pop("overflow_to_sample_mapping")
    # for key, values in dataset_row.items():
    #     tokenize[key] = [values[i] for i in sample_map]
    # return tokenize
    
token_datasets = dataset.map(tokenize_text,remove_columns=["text"])

  0%|          | 0/12050 [00:00<?, ?ex/s]

  0%|          | 0/268 [00:00<?, ?ex/s]

## Importing Models

In [4]:
from transformers import BertForSequenceClassification 
model = BertForSequenceClassification . \
        from_pretrained(load_path if load_model else bert_huggingface, num_labels=2)

# The warning is expected because we're importing from BertForPreTraining Model

In [6]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31923, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## Preparing Training

In [ ]:
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer

def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch",
                                  num_train_epochs=num_epoch)

trainer = Trainer(
    model,
    training_args,
    train_dataset=token_datasets["train"],
    eval_dataset=token_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
# Validation data => Seimbangin

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw. If raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12050
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4521


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.340900,0.727919,0.708955,0.715328
2,0.190700,0.999828,0.746269,0.748148
3,0.109900,1.323962,0.738806,0.738806


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

Saving model checkpoint to test-trainer/checkpoint-2000
Configuration saved in test-trainer/checkpoint-2000/config.json
Model weights saved in test-trainer/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-2000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-2500
Configuration saved in test-trainer/checkpoint-2500/config.json
Model weights saved in test-trainer/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-2500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-3000
Configuration saved in test-trainer/checkpoint-3000/config.json
Model weights saved in test-trainer/checkpoint-3000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-3000/tokenizer_config.json
Special tokens file 

TrainOutput(global_step=4521, training_loss=0.2294672842812575, metrics={'train_runtime': 1762.6577, 'train_samples_per_second': 20.509, 'train_steps_per_second': 2.565, 'total_flos': 4755732325632000.0, 'train_loss': 0.2294672842812575, 'epoch': 3.0})

In [ ]:
trainer.save_model(save_model_path)

Saving model checkpoint to /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet
Configuration saved in /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/config.json
Model weights saved in /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/special_tokens_map.json


# Predict Data

In [ ]:
model_path = "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet"
csv_out_data_path = "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/5. Predict Data/predicted/IndoBERTweet"

# Get All csv data paths
data = {
    "astra" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/2. Clean/Twitter/v3/astrazeneca.csv",
    "pfizer" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/2. Clean/Twitter/v3/pfizer.csv",
    "moderna" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/2. Clean/Twitter/v3/moderna.csv",
    "sinopharm" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/2. Clean/Twitter/v3/sinopharm.csv",
    "sinovac" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/2. Clean/Twitter/v3/sinovac.csv",
    "gratis" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/2. Clean/Twitter/v3/gratis.csv",
    "bayar" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/2. Clean/Twitter/v3/bayar.csv",
    "efektif" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/2. Clean/Twitter/v3/efektif.csv",
}

from transformers import AutoTokenizer, BertForSequenceClassification, Trainer
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)
model = Trainer(model)

# tokenize text
def tokenize_text_data(dataset_row):
    return tokenizer(str(dataset_row["clean"]), max_length=seq_length,
                     truncation=True, padding="max_length")
tokenized = load_dataset('csv', data_files=data)
tokenized = tokenized.map(tokenize_text_data)

# Predict all data
for i in data:
    data_path = data[i]
    df = pd.read_csv(data_path)

    model_prediction = model.predict(tokenized[i])
    logits = model_prediction.predictions
    softmax = tf.compat.v1.math.softmax(logits)
    out_ids = tf.math.argmax(softmax, axis=1).numpy()

    df["prediction"] = out_ids
    df.to_csv(f"{csv_out_data_path}/{i}.csv")

Didn't find file /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/added_tokens.json. We won't load it.
loading file /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/vocab.txt
loading file /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/tokenizer.json
loading file None
loading file /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/special_tokens_map.json
loading file /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/tokenizer_config.json
loading configuration file /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4.1 Train Test Baru/50 Manual/model/IndoBERTweet/config.json
Mode

Extracting data files:   0%|          | 0/8 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-aecdc5c78d1a998f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/1467 [00:00<?, ?ex/s]

...

  0%|          | 0/2975 [00:00<?, ?ex/s]

  0%|          | 0/1127 [00:00<?, ?ex/s]

  0%|          | 0/2020 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean, raw. If clean, raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1467
  Batch size = 8


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean, raw. If clean, raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2091
  Batch size = 8
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean, raw. If clean, raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1288
  Batch size = 8
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean, raw. If clean, raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 633
  Batch size = 8